In [12]:
import pandas as pd #To hand with data 
import numpy as np #To math 
import seaborn as sns #to visualization
import matplotlib.pyplot as plt # to plot the graphs
import matplotlib.gridspec as gridspec # to do the grid of plots

from sklearn.metrics import classification_report
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import precision_score, recall_score, fbeta_score, confusion_matrix, precision_recall_curve, accuracy_score
from sklearn.preprocessing import MinMaxScaler
from pathlib import Path
import librosa
from sklearn.pipeline import Pipeline

#SMOTE
from imblearn.pipeline import make_pipeline # To do our transformation in a unique time
from imblearn.over_sampling import SMOTE
from imblearn.metrics import classification_report_imbalanced

#models
from interpret.glassbox import ExplainableBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import StackingClassifier

In [13]:
# Source CSV
#df = pd.read_csv(r"../input/coughclassifier-trial/Smote_data.csv")
df = pd.read_csv(r"C:\Users\DELL\COV_Project\Pipeline\with_coswara_csv_smote.csv")

In [14]:
# 1 - Healthy
# 0 - COVID +ve
seed = 1

In [15]:
X = df.iloc[: ,:-1]
Y = df.iloc[: ,-1]

#Split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

features = ['chroma_stft', 'rmse', 'spectral_centroid', 'spectral_bandwidth',
       'rolloff', 'zero_crossing_rate', 'mfcc1', 'mfcc2', 'mfcc3', 'mfcc4',
       'mfcc5', 'mfcc6', 'mfcc7', 'mfcc8', 'mfcc9', 'mfcc10', 'mfcc11',
       'mfcc12', 'mfcc13', 'mfcc14', 'mfcc15', 'mfcc16', 'mfcc17', 'mfcc18',
       'mfcc19', 'mfcc20']

In [16]:
# For testing various models
# -----------------------------------------------------------------------------
# # get a list of models to evaluate
# def get_models():
# 	models = dict()
# 	models['rfc'] = RandomForestClassifier(random_state=1234)
# 	models['logreg'] =  LogisticRegression()
# 	models['clf'] = DecisionTreeClassifier(criterion="entropy", max_depth=5)
# 	models['XG Boost'] = XGBClassifier()
# 	models['ebm'] = ExplainableBoostingClassifier(random_state=seed)
# 	models['stacking'] = get_stacking()
# 	return models
 
# # evaluate a given model using cross-validation
# def evaluate_model(model, X, y):
# 	cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# 	scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score='raise')
# 	return scores

# def evaluate_model1(model, X, y):
# 	cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# 	scores1 = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# 	return scores1

# # get the models to evaluate
# models = get_models()
# # evaluate the models and store results
# results, names = list(), list()
# for name, model in models.items():
# 	scores = evaluate_model(model, X, Y)
# 	scores1 = evaluate_model1(model, X, Y)
# 	results.append(scores)
# 	names.append(name)
# 	print('%s: %f, %f, %f' % (name, mean(scores1), mean(scores), std(scores)))
# 	print(scores)
# 	print(scores1)
# 	print("-----------------------------------------")
# # plot model performance for comparison
# pyplot.boxplot(results, labels=names, showmeans=True)
# pyplot.show()

## Ensemble Model

In [17]:
# make a prediction with a stacking ensemble
# define the base models
level0 = list()
level0.append(('rfc', RandomForestClassifier(random_state=1234)))
level0.append(('logreg', LogisticRegression()))
level0.append(('XG Boost', XGBClassifier()))
level0.append(('ebm', ExplainableBoostingClassifier(random_state=seed)))
level0.append(('clf',DecisionTreeClassifier(criterion="entropy", max_depth=5)))
# define meta learner model
level1 = LogisticRegression()
# define the stacking ensemble
model = StackingClassifier(estimators=level0, final_estimator=level1, cv=5)
# fit the model on all available data
model.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:56:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:56:22] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:56:24] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:56:24] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


StackingClassifier(cv=5,
                   estimators=[('rfc',
                                RandomForestClassifier(random_state=1234)),
                               ('logreg', LogisticRegression()),
                               ('XG Boost',
                                XGBClassifier(base_score=None, booster=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None, gamma=None,
                                              gpu_id=None,
                                              importance_type='gain',
                                              interaction_constraints=None,
                                              learning_rate=None,
                                              max_delta_step=No...
                                              n_estimators=100, n_jobs=None,
                                       

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
X, y = make_classification(random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    random_state=0)
pipe = Pipeline([('scaler', StandardScaler()), ('lr', LogisticRegression())])
# The pipeline can be used as any other estimator
# and avoids leaking the test set into the train set
pipe.fit(X_train, y_train)

pipe.score(X_test, y_test)

0.92

In [8]:
import sklearn; sklearn.show_versions()


System:
    python: 3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]
executable: C:\ProgramData\Anaconda3\python.exe
   machine: Windows-10-10.0.19041-SP0

Python dependencies:
          pip: 21.1.2
   setuptools: 52.0.0.post20210125
      sklearn: 0.24.2
        numpy: 1.19.5
        scipy: 1.5.4
       Cython: 0.29.21
       pandas: 1.1.4
   matplotlib: 3.4.2
       joblib: 1.0.1
threadpoolctl: 2.1.0

Built with OpenMP: True


In [11]:
X_test.head()

AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [18]:

model.score(X_test,y_test)

0.9638418079096045

In [19]:
#model.score(X_test,y_test)
predictions = model.predict(X_test)
print(accuracy_score(y_test, predictions))
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

0.9638418079096045
[[438  15]
 [ 17 415]]
              precision    recall  f1-score   support

           0       0.96      0.97      0.96       453
           1       0.97      0.96      0.96       432

    accuracy                           0.96       885
   macro avg       0.96      0.96      0.96       885
weighted avg       0.96      0.96      0.96       885



## Save to Pickle File

In [20]:
import pickle
filename = 'ensemble_model.sav'
pickle.dump(model, open(filename, 'wb'))

### Model Evaluation

In [21]:
# # Evaluate predictions
# print(accuracy_score(y_test, predictions))
# print(confusion_matrix(y_test, predictions))
# print(classification_report(y_test, predictions))

# # Importing the metrics package from sklearn library
# from sklearn import metrics
# # Creating the confusion matrix
# cm = metrics.confusion_matrix(y_test, y_pred)
# # Assigning columns names
# cm_df = pd.DataFrame(cm, 
#             columns = ['Predicted COVID+ve', 'Predicted Healthy'],
#             index = ['Actual COVID+ve', 'Actual Healthy'])
# # Showing the confusion matrix
# cm_df

# # Making the Confusion Matrix
# from sklearn.metrics import confusion_matrix
# from sklearn.metrics import classification_report
# clf_rpt = classification_report(y_test,y_pred)
# print("Accuracy =", accuracy_score(y_test, y_pred))
# print(clf_rpt)
# cm = confusion_matrix(y_test, y_pred)
# sns.heatmap(cm,annot=True)

In [22]:
# import eli5
# # create our dataframe of feature importances
# feat_imp_df = eli5.explain_weights_df(rfc, feature_names=features)
# feat_imp_df

In [23]:
# Code for converting new audio to required format

path = r'C:\Users\DELL\COV_Project\Files\cough-heavy.wav'
def preproces(fn_wav):
    y, sr = librosa.load(fn_wav, mono=True, duration=5)
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    rmse = librosa.feature.rms(y=y)
    spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    zcr = librosa.feature.zero_crossing_rate(y)
    mfcc = librosa.feature.mfcc(y=y, sr=sr)
    
    feature_row = {        
        'chroma_stft': np.mean(chroma_stft),
        'rmse': np.mean(rmse),
        'spectral_centroid': np.mean(spectral_centroid),
        'spectral_bandwidth': np.mean(spectral_bandwidth),
        'rolloff': np.mean(rolloff),
        'zero_crossing_rate': np.mean(zcr),        
    }
    for i, c in enumerate(mfcc):
        feature_row[f'mfcc{i+1}'] = np.mean(c)
    
    return feature_row

def get_dataframe(feature_row):
    data = pd.DataFrame.from_dict(feature_row, orient='index')
    data2 = data.T
    return data2

feature = preproces(path)
# data = pd.DataFrame.from_dict(feature, orient='index')
# data2 = data.T

def scaler_transform(feature):
    df = pd.read_csv(r"C:\Users\DELL\COV_Project\Files\smote_no_encode.csv") #Source File
    scaler = MinMaxScaler()
    X = scaler.fit_transform(np.array(df.iloc[:, :-1]))
    X_s = pd.DataFrame(X, columns = features)
    X_s['label'] = df['label']
    test_normalised = scaler.transform(feature)
    return test_normalised

def to_scaled_feature(path):
    retro = preproces(path)
    retro1 = get_dataframe(retro)
    test_data = scaler_transform(retro1)
    return test_data   

In [24]:
def smote_preprocess(old_file, new_file):
    df_old = pd.read_csv(old_file)
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2, test_size=0.30)
    # build model with SMOTE imblearn
    smote_pipeline = make_pipeline_imb(SMOTE(random_state=4), \
                                   classifier(random_state=42))
    smote_model = smote_pipeline.fit(X_train, y_train)
    #Showing the diference before and after the transformation used
    print("normal data distribution: {}".format(Counter(y)))
    X_smote, y_smote = SMOTE().fit_sample(X, y)
    print("SMOTE data distribution: {}".format(Counter(y_smote)))
    X_s = pd.DataFrame(X_smote, columns = features)
    y_s = pd.DataFrame(y_smote, columns = ['label'])
    X_s['label'] = y_s
    X_s.to_csv(new_file, index=False)
    return new_file

In [2]:
pip install ffmpeg

  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6083 sha256=82113cfd5f10e0c390951ee8a54071a7b1e5c12d6ad41cd42fd3babb6873b514
  Stored in directory: c:\users\dell\appdata\local\pip\cache\wheels\30\33\46\5ab7eca55b9490dddbf3441c68a29535996270ef1ce8b9b6d7
Successfully built ffmpeg
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os,sys
from pydub import AudioSegment

In [5]:
orig_song = r"C:\Users\DELL\COV_Project\Files\virufy-cdf-coughvid_0007c6f1-5441-40e6-9aaf-a761d8f2da3b.webm" # Name of the person.ogg
dest_song = 'file_example_OOG_5MG.wav'#orig_song
#dest_song1 = orig_song.split(", ")[0]

def convert_webm_to_wav():
    song = AudioSegment.from_ogg(orig_song)
    song.export(dest_song, format="wav")
    
if __name__ == '__main__':
    convert_webm_to_wav()

C:\ProgramData\Anaconda3\lib\site-packages\pydub\utils.py:198: RuntimeWarning: Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work
  warn("Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work", RuntimeWarning)


FileNotFoundError: [WinError 2] The system cannot find the file specified

In [7]:
ffmpeg -i orig_song -crf 23 "out.wav"

SyntaxError: invalid syntax (<ipython-input-7-b936fa9a6aa2>, line 1)